In [1]:
#polly - synthesize speech (text to speech)

In [2]:
import boto3


In [4]:
polly_client = boto3.client('polly',region_name='us-east-1')

In [10]:
text = "Hi, this is a sample text created by Bhargava"

In [11]:
response = polly_client.synthesize_speech(Engine='standard',
                                         Text = text,
                                         OutputFormat = 'mp3',
                                         VoiceId='Matthew')

In [12]:
with open('speech.mp3','wb') as file:
          file.write(response['AudioStream'].read())

In [ ]:
#Transcribe   (use case: call analytics, subtitles and meeting notes, toxicity detection)

In [14]:
client = boto3.client('s3')

In [15]:
client.create_bucket(Bucket='demo-bucket-transribe')

{'ResponseMetadata': {'RequestId': 'QZCND4PVC1D3KQWV',
  'HostId': 'f7ycS/XHCgieSW8z0cDw5ECs7XrIhqh/BXkCBCtYD21Mp1qNlwRjw/zdzn5Ygy9mRR0NaGoxIIvSKwKVPMJUAZRP7JghLAny',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'f7ycS/XHCgieSW8z0cDw5ECs7XrIhqh/BXkCBCtYD21Mp1qNlwRjw/zdzn5Ygy9mRR0NaGoxIIvSKwKVPMJUAZRP7JghLAny',
   'x-amz-request-id': 'QZCND4PVC1D3KQWV',
   'date': 'Tue, 31 Oct 2023 04:13:58 GMT',
   'location': '/demo-bucket-transribe',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/demo-bucket-transribe'}

In [17]:
client.upload_file(Filename = 'speech.mp3',Bucket='demo-bucket-transribe',Key = 'new-speech-file')

In [22]:
transcribe_client = boto3.client('transcribe',region_name='us-east-1')
job_name = 'Bhargava_transribe'

In [25]:
transcribe_client.start_transcription_job(TranscriptionJobName=job_name,
                                         Media = {'MediaFileUri':"s3://demo-bucket-transribe/new-speech-file"},
                                         MediaFormat='mp3',
                                         LanguageCode='en-US')

ConflictException: An error occurred (ConflictException) when calling the StartTranscriptionJob operation: The requested job name already exists. Use a different job name.

In [28]:
#Implementing own waiter so that 


import time
while True:
    status = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED','FAILED']:
        break

In [31]:
status

{'TranscriptionJob': {'TranscriptionJobName': 'Bhargava_transribe',
  'TranscriptionJobStatus': 'COMPLETED',
  'LanguageCode': 'en-US',
  'MediaSampleRateHertz': 22050,
  'MediaFormat': 'mp3',
  'Media': {'MediaFileUri': 's3://demo-bucket-transribe/new-speech-file'},
  'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/732077674619/Bhargava_transribe/62884082-88c5-4e66-b375-5b6b8d9e4f4d/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEMT%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIEeRhxpCsxS9NiExZ7ysmwcBUqVGP02TSUGkrKLqmdvFAiBARc49EqEgn35wPbZI4TVlHFXAluF0AyKCXA5aNuYvxiq6BQjt%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAQaDDI3NjY1NjQzMzE1MyIMHO6tZcViT20KqthmKo4FLFseIJ69I6xgF1IAjpYStFPmrR4hGazrxokq5DWZqN%2Fiu5gvIwwqSM7e%2BsPDIctwWikEdTdGFVI6o7SoQFYcdXvtGUZMnTGKHWm4cQL77FP2GYlhOSxw%2FzBogFDxxtxBbujTehDEn%2FXHxNWobDlC%2B4AX%2BYR90w7mC2pNA0%2Fa%2F517DyYadq71zuc0FKSRtNGbk5BN%2B6%2BSoAiGotwCvrfff%2Bayk4YFR7DN%2FHJyjPToK18Jd5YEBYLRypRf7Y7juhG

In [52]:
import requests
response = requests.get(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
with open('transribed.json','wb') as f:
    f.write(response.content)

In [65]:
import json
with open('transribed.json','r') as f:
#     print(f.read())
    data = json.load(f)

full_text = data['results']['transcripts'][0]['transcript']
full_text

'Hi, this is a sample text created by Barga.'

In [64]:
#AWS translate
translate = boto3.client('translate',region_name='us-east-1')

In [68]:
response = translate.translate_text(Text=full_text,
                                   SourceLanguageCode='en',
                                   TargetLanguageCode='mr')

In [69]:
response

{'TranslatedText': 'हाय, हा बर्गाने तयार केलेला एक नमुना मजकूर आहे.',
 'SourceLanguageCode': 'en',
 'TargetLanguageCode': 'mr',
 'ResponseMetadata': {'RequestId': 'b691ece3-c9f7-484d-84ff-97cebac2b4c2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b691ece3-c9f7-484d-84ff-97cebac2b4c2',
   'cache-control': 'no-cache',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '194',
   'date': 'Tue, 31 Oct 2023 13:38:29 GMT'},
  'RetryAttempts': 0}}

In [71]:
#Textract (OCR-optical character recognition)
textract_client= boto3.client('textract',region_name='us-east-1')
document_path = 'Test.pdf'

In [73]:
response = textract_client.detect_document_text(Document={'Bytes':open(document_path,'rb').read()})
response

{'DocumentMetadata': {'Pages': 1},
 'Blocks': [{'BlockType': 'PAGE',
   'Geometry': {'BoundingBox': {'Width': 1.0,
     'Height': 0.9999853372573853,
     'Left': 0.0,
     'Top': 1.4679527339467313e-05},
    'Polygon': [{'X': 0.0, 'Y': 1.4679527339467313e-05},
     {'X': 1.0, 'Y': 0.0007638687966391444},
     {'X': 1.0, 'Y': 1.0},
     {'X': 0.0, 'Y': 0.9997200965881348}]},
   'Id': 'a9d0c98c-c754-40de-8a37-47db1142bc79',
   'Relationships': [{'Type': 'CHILD',
     'Ids': ['32ac880d-378e-40aa-a0e9-186e0996401c',
      '42697ba5-71a4-4907-8fb0-843d2f343379',
      '3e3e5753-81f5-4223-9158-5b71e8492c34',
      '43e54701-292d-453c-9b2b-e4c5b6999a1e',
      'c8c02297-d62d-42f8-8dcf-70d9aa0fd40e',
      'e5db3014-7cf7-4790-8a89-f54950d320bd',
      '9ac8ffae-d44e-440e-b714-cb84b1df1bdb',
      'de05f5a8-8dbe-4423-8599-b9f2e829ddca',
      '0b8943c4-9676-4941-ab8c-8af21fb58f36',
      '17f38ad5-0829-4a5f-850b-35baedf7e97e',
      '0f146af6-c253-42e2-9d5a-337bb6bb3c16',
      '768c2bb4-ad9d-

In [80]:
text=''
for item in response['Blocks']:
    if item['BlockType'] in ['WORD']:
        text += item['Text'] + '\n'

In [81]:
print(text)

1
Sample
Text
Here
is
some
sample
text
with
a
dummy
table:
Column1
Column
2
Column
3
Test
1
Test
2
Test
3
Test
4
Test
5
Test
6
2
John
Doe
Here
is
a
phrase
about
John
Doe:
Name:
John
Doe
Age:
30
1



In [83]:
response = textract_client.analyze_document(Document={'Bytes':open(document_path,'rb').read()},
                                           FeatureTypes=['QUERIES'],
                                           QueriesConfig={
                                               'Queries':[{'Text':'what is the name of 30 year old guy?'}]
                                           })

response

{'DocumentMetadata': {'Pages': 1},
 'Blocks': [{'BlockType': 'PAGE',
   'Geometry': {'BoundingBox': {'Width': 1.0,
     'Height': 0.9999853372573853,
     'Left': 0.0,
     'Top': 1.4679526429972611e-05},
    'Polygon': [{'X': 0.0, 'Y': 1.4679526429972611e-05},
     {'X': 1.0, 'Y': 0.0007638687966391444},
     {'X': 1.0, 'Y': 1.0},
     {'X': 0.0, 'Y': 0.99972003698349}]},
   'Id': '9fb66afd-97d5-4bde-8f23-84424f186914',
   'Relationships': [{'Type': 'CHILD',
     'Ids': ['4d78a293-5d5f-4909-85d6-0d54bbd37da0',
      '59051072-6149-4c54-8604-e5f6f6adde57',
      '33153562-8df1-4474-9966-77829491be77',
      'fe091888-aa06-4216-a568-43d3db4e4588',
      '7aafb0b5-c012-4204-a0a8-2632d180b03d',
      '972d076a-c771-44db-aee8-1124a74ccd88',
      '11fbb896-595a-4159-9569-4c4803e0d133',
      '2f48c381-89e8-469f-939b-0b3e0f266805',
      'f3350cf6-d80c-48f4-8b7f-d382fed25abf',
      '55d9676c-221d-4b43-ac5c-1eae03bbe9a1',
      '7a79d1f3-e557-44c6-87f9-d729f005acfc',
      '2ab2e84e-9eed-4c

In [87]:
for block in response['Blocks']:
    if block['BlockType'] == 'QUERY_RESULT':
        print(block['Text'])

John Doe
30
